In [1]:
import pandas as pd
import numpy as np
from dbfread import DBF
import pickle

In [2]:
def load_dbf_to_df(file_name):
    dbf = DBF(data_path + file_name, load=True, raw=True)
    df = pd.DataFrame(dbf.records)
    return df.applymap(lambda x: x.rstrip())

In [3]:
#pickle objects
def make_pickle(obj, filename):
    with open(filename, 'wb') as picklefile:
        pickle.dump(obj, picklefile)

#open pickled objects
def open_pickle(filename):
    with open(filename, 'rb') as picklefile:
        return pickle.load(picklefile)

### MassGIS Public Water Sources

In [4]:
data_path = '../data/external/MassGIS/rs101416/'

** PWSDEP_PT_WQTS **

The PWS_DEP.WQTS table contains information downloaded from the DWP Water Quality Testing System (WQTS) Oracle database. WQTS is DEP’s comprehensive repository for PWS information. The WQTS database contains many other data items that are not in the PWSDEP_PT_WQTS relate file. This table was designed for use by the DEP GIS Program to maintain the PWS datalayer in close relation to WQTS. 

In [5]:
file_name = 'PWSDEP_PT_WQTS.dbf'
pws = load_dbf_to_df(file_name)

In [6]:
pws.to_csv('../data/interim/pws_wqts.csv')

** PWSDEP_PT **

The DEP PWS datalayer has been compiled from several sources. The original DEP PWS point dataset contained only community water supplies (CWS) which were located and digitized from stable mylar overlays based on USGS topographic quadrangles. In 1993 locations for community and non-community sources were generated from global positioning system (GPS) data collected by the U.S. Environmental Protection Agency (EPA). The EPA GPS locations were merged with the existing DEP CWS point data to create the DEP PWS datalayer. From June 1996 to February 1997 the DEP GIS Program and Drinking Water Program (DWP) conducted an intensive update of the PWS datalayer. This update involved several methodologies including map interpolation and GPS field verification.

In [7]:
file_name = 'PWSDEP_PT.dbf'
pts = load_dbf_to_df(file_name)

In [8]:
pws['PWS_CLASS'].value_counts()

COM     2550
NC      2300
NTNC     681
Name: PWS_CLASS, dtype: int64

In [9]:
pts['TYPE'].value_counts()

GW      1708
TNC     1490
NTNC     410
SW       192
ESW       70
PW        50
WF         1
Name: TYPE, dtype: int64

In [10]:
#add public water supply data to each point
pts = pts.merge(pws, how='left', on='SOURCE_ID')

In [11]:
#drop proposed wells
pts = pts[pts['TYPE'] != 'PW']

Create dataframe of potential public water sources for schools, which could be:
* town-wise system
* town system specific to school

In [12]:
#PWS related to town water departments are all COM, with the following keywords in PWS_NAME
towns = pts['TOWN'].tolist()
for town in ['FOXBORO', 'CENTERVILLE', 'BARNSTABLE', 'COTUIT', 'ABINGTON', 'ROCKLAND']:
    towns.append(town)
muni_keywords = ['DEP', 'DPW', 'DIST', 'WORKS', 'DIV', 'COMM.', 'COMMISSION', 'SUPPLY', 'WD' 'SYS', 'WORKS', 'TOWN OF']

In [13]:
#mark PWS with muni keywords in PWS_NAME as potential school water supply (['SCH_SUPPLY']=1)
pts.loc[pts['PWS_NAME'].str.contains('|'.join(towns)) & pts['PWS_NAME'].str.contains('|'.join(muni_keywords)), 'SCH_SUPPLY'] = 1

In [14]:
#mark PWS with 'SCHOOL' in PWS_NAME as potential school water supply (['SCH_SUPPLY']=1)
pts.loc[pts['PWS_NAME'].str.contains('SCHOOL'), 'SCH_SUPPLY'] = 1

In [15]:
school_pts = pts[pts['SCH_SUPPLY'] == 1]

Create dataframe with one row for each PWS_ID/TOWN that includes:
* PWS_ID
* PWS_NAME
* TOWN
* PWS_CLASS
* COUNT_GW (# of groundwater intakes in system)
* COUNT_SW (# of surface water intakes in system)

In [16]:
#drop emergency surface water (ESW) pts and well field (WF)
school_pts = school_pts[school_pts['TYPE'] != 'ESW']
school_pts = school_pts[school_pts['TYPE'] != 'WF']

#keep only:
#active suppliers (PWS_STATUS = A)
#active Sources (S_STATUS = A)
#available Sources (S_AVAILABI = ACTIVE)
school_pts = school_pts[school_pts['PWS_STATUS'] == 'A']
school_pts = school_pts[school_pts['S_STATUS'] == 'A']
school_pts = school_pts[school_pts['S_AVAILABI'] == 'ACTIVE']

In [17]:
#find type for each intake based on SOURCE_ID
school_pts['INTAKE_TYPE'] = school_pts['SOURCE_ID'].apply(lambda x: x[-1])

In [18]:
sch_pws = school_pts[['PWS_ID', 'PWS_NAME', 'TOWN', 'PWS_CLASS']].drop_duplicates()

In [19]:
#calculate number of gw and sw intakes for each system
intakes = school_pts.groupby(['PWS_ID', 'INTAKE_TYPE']).count()[['PWS_NAME']]
intakes = intakes.unstack()
intakes.fillna(0.0, inplace=True)
intakes.reset_index(inplace=True)

In [20]:
sch_pws = sch_pws.merge(intakes, on='PWS_ID')

/Users/evaward/anaconda/lib/python2.7/site-packages/pandas/tools/merge.py:205: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
/Users/evaward/anaconda/lib/python2.7/site-packages/pandas/tools/merge.py:478: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  self.right = self.right.drop(right_drop, axis=1)


In [21]:
columns = 'PWS_ID', 'PWS_NAME', 'TOWN', 'PWS_CLASS', 'COUNT_GW', 'COUNT_SW'
sch_pws.columns = columns

Create GW ratio column

In [22]:
sch_pws['TOTAL_INTAKES'] = sch_pws[['COUNT_GW', 'COUNT_SW']].sum(axis=1)

In [23]:
sch_pws['GW_RATIO'] = sch_pws['COUNT_GW'].div(sch_pws['TOTAL_INTAKES'])

In [24]:
sch_pws.drop(['COUNT_GW', 'COUNT_SW'], axis=1, inplace=True)

In [25]:
sch_pws.head()

,PWS_ID,PWS_NAME,TOWN,PWS_CLASS,TOTAL_INTAKES,GW_RATIO
0,2141000,HUDSON WATER SUPPLY,HUDSON,COM,6.0,0.833333
1,1349000,WORTHINGTON FIRE DISTRICT,WORTHINGTON,COM,10.0,1.000000
2,3023000,BEDFORD WATER DEPT.,BEDFORD,COM,3.0,1.000000
3,3067000,CONCORD WATER DEPT,CONCORD,COM,10.0,0.900000
4,3198000,NATICK WATER DEPARTMENT,NATICK,COM,10.0,1.000000


### Add MWRA systems


In [26]:
sch_pws['MWRA'] = 0

In [27]:
mwra = open_pickle('../data/interim/mwra.pkl')

In [28]:
sch_pws = pd.concat([sch_pws, mwra])

### Add # of service connections from PWS contact info csv

In [29]:
connections = pd.read_csv('../data/external/MassDEP/pwscont.csv',
                          usecols=['PWSID', '# SERVICE CONNECTIONS'], 
                          dtype='str')

connections.columns = ['PWS_ID', 'COUNT_CONN']

In [30]:
sch_pws = sch_pws.merge(connections, on='PWS_ID')

In [31]:
len(sch_pws)

386

### Add results and whether reported pb90 results from pb90 csv

In [32]:
pb90 = pd.read_csv('../data/external/MassDEP/pb90.csv', 
                   usecols=['PWSID', 'RESULT'],
                   dtype='str')
pb90.columns = ['PWS_ID', 'PB90_RESULT']

In [33]:
pb90.head()

,PWS_ID,PB90_RESULT
0,1013000,0.0087
1,1013001,0
2,1022017,0.0037
3,1022026,0.0006
4,1024000,0.00702


In [34]:
sch_pws = sch_pws.merge(pb90, how='left', on='PWS_ID')

In [35]:
sch_pws.isnull().sum()/len(sch_pws)

GW_RATIO         0.000000
MWRA             0.000000
PWS_CLASS        0.000000
PWS_ID           0.000000
PWS_NAME         0.000000
TOTAL_INTAKES    0.000000
TOWN             0.000000
COUNT_CONN       0.000000
PB90_RESULT      0.443005
dtype: float64

In [36]:
sch_pws['PB90_RESULT'].fillna(-1, inplace=True)

In [37]:
sch_pws.loc[sch_pws['PB90_RESULT'] >= 0, 'PB_90'] = 1
sch_pws.loc[sch_pws['PB90_RESULT'] < 0, 'PB_90'] = 0

In [38]:
sch_pws.head()

,GW_RATIO,MWRA,PWS_CLASS,PWS_ID,PWS_NAME,TOTAL_INTAKES,TOWN,COUNT_CONN,PB90_RESULT,PB_90
0,0.833333,0,COM,2141000,HUDSON WATER SUPPLY,6.0,HUDSON,5819,0,1.0
1,1.000000,0,COM,1349000,WORTHINGTON FIRE DISTRICT,10.0,WORTHINGTON,170,0.0036,1.0
2,1.000000,0,COM,3023000,BEDFORD WATER DEPT.,3.0,BEDFORD,4399,-1,0.0
3,0.900000,0,COM,3067000,CONCORD WATER DEPT,10.0,CONCORD,5704,-1,0.0
4,1.000000,0,COM,3198000,NATICK WATER DEPARTMENT,10.0,NATICK,10967,-1,0.0


In [39]:
#sch_pws.drop('RESULT', axis=1, inplace=True)

### Add whether responded to LCR survey from lcrsurvresp111616 csv

In [40]:
survey = pd.read_csv('../data/external/MassDEP/lcrsurvresp111616.csv', 
                   usecols=['Unnamed: 1', 'Unnamed: 2'],
                   dtype='str')
survey.columns = ['PWS_ID', 'SURVEY']

In [41]:
survey.drop(0, inplace=True)

In [42]:
survey['SURVEY'] = 1

In [43]:
sch_pws = sch_pws.merge(survey, how='left', on='PWS_ID')
sch_pws['SURVEY'].fillna(0, inplace=True)

In [44]:
sch_pws.isnull().sum()

GW_RATIO         0
MWRA             0
PWS_CLASS        0
PWS_ID           0
PWS_NAME         0
TOTAL_INTAKES    0
TOWN             0
COUNT_CONN       0
PB90_RESULT      0
PB_90            0
SURVEY           0
dtype: int64

In [45]:
make_pickle(sch_pws, '../data/interim/sch_pws.pkl')